In [3]:

# TODO: keep working on pruning

interactive!


In [1]:
# dev pruning

#!/usr/bin/env python3
"""
stream data to a Pub/Sub topic
"""

def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')


# merge seconds -> minutes
# update seconds -> minutes
# partial aggregation

# logarithmic publishing (every 12 seconds; 5 hours, 6 days, etc.)
# original publish at finest scale
# select, bin, publish, delete
# add old data if appropriate
# when? every agregation_label[1]
# verify that we are not agregating twice. 
# 10 trips in each bin

# print("adding pruned_test data")
# aggregation_lengths = [120, 120, 48, 60, 24, 10]
# aggregation_labels = (
#     ["seconds", "minutes", "hours", "days", "months", "years"])
# date0 = datetime(2021, 10, 31)

# min/max object lists not updating correctly

# dev packages:
import sys

# prod packages
import os
import distogram
from datetime import datetime
import jsonpickle
import time
import pandas as pd

from dateutil.relativedelta import relativedelta

from sqlalchemy import text
from sqlalchemy.orm import sessionmaker

try:
    from icecream import ic
except ImportError:  # Graceful fallback if IceCream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa


from common import (
    Base, return_test_engine, LabelledDistogram, make_distribution, 
    make_distogram, AggregationType, delete_tables, taxi_data_headers_map)

debug = False

try:
    import zoneinfo
except ImportError:
    from backports import zoneinfo

    
    
project_id = os.environ.get('DEVSHELL_PROJECT_ID')
dataset = 'default_dataset'
sleep = 1.0  # seconds
batch_size = 10000

# store histograms, time, table, project, 


def prune_test_data(engine):
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    aggregation_lengths = [120, 120, 48, 60, 24, 10]
    aggregation_labels = (
        ["seconds", "minutes", "hours", "days", "months", "years"])

    # load initial data
    labelled_distogram_list = list(
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.datetime)) 
    labelled_distogram_hash = {
        instance.primary_key: instance for instance in labelled_distogram_list}

    df_metadata = pd.DataFrame(
        [instance.metadata_list for instance in labelled_distogram_list],
        columns=labelled_distogram_list[0].metadata_labels)

    ic(df_metadata.columns)
    if is_interactive():
#         display(df_metadata)
        pass
    first_time = df_metadata.iloc[0].at['datetime']
    last_time = df_metadata.iloc[-1].at['datetime']
    df_metadata['parsed'] = False
    
    # combine seconds to minutes
    # @TODO: writing time rounding function
    aggregation_index = 0
    df_fine = df_metadata[
        df_metadata['aggregation_type'] == aggregation_labels[aggregation_index]]
    
    delta_dict = {
        aggregation_labels[aggregation_index]: aggregation_lengths[aggregation_index]}
    min_keep_time = last_time - relativedelta(**delta_dict)

    delta_dict = {aggregation_labels[aggregation_index + 1]: 1}

    merge_stop_time = last_time + relativedelta(seconds = 1)
#     while merge_stop_time > first_time:
#     if True:
#     ic(len(df_fine))
    for i in range(10):
        merge_start_time = merge_stop_time - relativedelta(**delta_dict)

        rounding_dict = {"microsecond": 0}
        for j in range(aggregation_index + 1):
            rounding_dict[aggregation_labels[j][:-1]] = 0
        merge_start_time = merge_start_time.replace(**rounding_dict)
#         ic(i)
#         ic(merge_start_time)
#         ic(merge_stop_time)
        
        df_mask = ((df_fine["datetime"] < merge_stop_time)
                    & (df_fine["datetime_min"] >= merge_start_time))
        df_merge = df_fine[df_mask]
        df_fine.loc[df_mask, "parsed"] = True
m        ic(len(df_merge))
        ic(df_merge)
        if is_interactive():
            pass
#             display(df_merge)
        if len(df_mask) > 0:
            merge_stop_time = merge_start_time
#             ic()
        else:
            merge_stop_time = df_fine[df_fine["parsed"] == False].iloc[-1].at['datetime']
            merge_stop_time = merge_stop_time + relativedelta(seconds = 1)
            ic()
            
#     df_merge = df_metadata[(
#         df_metadata['datetime']]

# df1 = df[(df.a != -1) & (df.b != -1)]

def publish_pruned_test_data(engine):
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    if True:
        print("adding pruned_test data")
        aggregation_lengths = [120, 120, 48, 60, 24, 10]
        aggregation_labels = (
            ["seconds", "minutes", "hours", "days", "months", "years"])
        date0 = datetime(2021, 10, 31)
        for i in range(len(AggregationType)):
        # i = 0
        # if True:
            print(AggregationType(i).name)
            j_min = 0
            j_max = 5 * aggregation_lengths[i]
            j_delta = aggregation_lengths[i] // 5
            last_date = date0
            for j in range(j_min, j_max, j_delta):
                delta_dict = {aggregation_labels[i]: -j}
                delta = relativedelta(**delta_dict)
                date = date0 + delta
                # America/New_York
                date.replace(tzinfo=zoneinfo.ZoneInfo('Etc/UTC'))
                h = make_distogram(make_distribution())
                d = LabelledDistogram(
                    data_source="debug3",
                    variable_name="x",
                    datetime_min=last_date,
                    datetime=date,
                    aggregation_type=AggregationType(0).name,
                    temporary_record=False,
                    distogram=h)
                session.add(d)
                session.commit()
                last_date = date
        print("before commit")
        session.commit()
        print("after commit")


def publish_taxi_data(engine):

    def publish_single_time():
        for header, my_distogram in distogram_dict.items():
            d = LabelledDistogram(
                data_source="taxi_min_max",
                variable_name=header,
                datetime=max_datetime,
                aggregation_type=aggregation_type,
                distogram=my_distogram)
            session.add(d)
            session.commit()
        print(f"published data: {max_datetime}")
        print(f"data_source: {d.data_source}")   
             
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    print("taxi!")
    # minutes/hours/days/weeks
    df = pd.read_parquet('data.parquet')
    df.sort_values(by=['tpep_dropoff_datetime'])
    # df2 = df.tail(1000000)['dropoff_latitude']
    # df2.to_csv('latitude.csv')
    # sys.exit()
    aggregation_dict = {
        # "minutes": 15, 
        # "hours": 1, 
        "days": 1}
    for aggregation_type, aggregation_length in aggregation_dict.items():

        delta_time_dict = {aggregation_type: aggregation_length}
        delta_time = relativedelta(**delta_time_dict)
        dt0 = df.iloc[0]['tpep_dropoff_datetime']
        print(f"dt0: {dt0}")
        if aggregation_type in {"minutes", "hours"}:
            dt0 = datetime(*dt0.timetuple()[:4])  # year/month/day/hour
        else:
            dt0 = datetime(*dt0.timetuple()[:3])  # year/month/day
        print(f"dt0: {dt0}")
        min_datetime = dt0
        max_datetime = dt0 + delta_time
        create_new_distograms = True
        for index, row in df.tail(1000000).iterrows():
            if create_new_distograms:
                distogram_dict = {
                    header: distogram.Distogram(with_min_max_list=True) 
                    for header in taxi_data_headers_map
                    if header != 'tpep_dropoff_datetime'}
                create_new_distograms = False
            for header, my_distogram in distogram_dict.items():
                if debug:
                    print(f"json pickle")
                    # print(f"empty: {jsonpickle.encode(my_distogram, indent=2)}")
                    my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                    # print(f"None max_min: {jsonpickle.encode(my_distogram, indent=2)}")
                    my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                    print(f"Row max_min: {jsonpickle.encode(my_distogram, indent=2)}")
                    # print(f"Row.to_dict: {jsonpickle.encode(row.to_dict(), indent=2)}")
                    sys.exit()
                # my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
            if row['tpep_dropoff_datetime'] > max_datetime:
                create_new_distograms = True 
                publish_single_time()
                min_datetime = max_datetime
                max_datetime = max_datetime + delta_time
        publish_single_time()


def main(engine):

    print("before create")
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    print("before read")
    success = False
    for instance in (
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.primary_key)):
        success = True
        # print(instance.primary_key, instance.variable_name, instance.datetime)


    if success:
        print()
        print(instance.primary_key, instance.variable_name)
        h2 = jsonpickle.decode(instance.distogram_string)
        print(f"min/mean/max {h2.min}/{distogram.mean(h2)}/{h2.max}")



    # publish_taxi_data(engine)
    # every 
    print("read from database")
    for instance in (
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.primary_key)):
        pass
    print(
        instance.primary_key,
        instance.variable_name,
        instance.aggregation_type,
        instance.data_source,
        instance.datetime)
    
        # publish_pruned_test_data(engine)
#     prune_test_data(engine)

# from datetime import date
# from dateutil.relativedelta import relativedelta

# six_months = date.today() + relativedelta(months=+6)


if __name__ == "__main__":
    import os
    database_list = [
        "bigquery", "sqlite-memory", "sqlite-disk", "sqlite-disk-2", "postgres"]
    database = database_list[3]
    engine = return_test_engine(database)
    print(engine)

    # test(engine)
#     main(engine)
#     os.remove("./localdb-2")
#     publish_pruned_test_data(engine)

    prune_test_data(engine)


SyntaxError: invalid syntax (3793255570.py, line 135)

In [12]:
#!/usr/bin/env python3
"""
stream data to a Pub/Sub topic
"""

def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')


# merge seconds -> minutes
# update seconds -> minutes
# partial aggregation

# logarithmic publishing (every 12 seconds; 5 hours, 6 days, etc.)
# original publish at finest scale
# select, bin, publish, delete
# add old data if appropriate
# when? every agregation_label[1]
# verify that we are not agregating twice. 
# 10 trips in each bin

# print("adding pruned_test data")
# aggregation_lengths = [120, 120, 48, 60, 24, 10]
# aggregation_labels = (
#     ["seconds", "minutes", "hours", "days", "months", "years"])
# date0 = datetime(2021, 10, 31)

# min/max object lists not updating correctly

# dev packages:
import sys

# prod packages
import os
import distogram
from datetime import datetime
import jsonpickle
import time
import pandas as pd

from dateutil.relativedelta import relativedelta

from sqlalchemy import text
from sqlalchemy.orm import sessionmaker

try:
    from icecream import ic
except ImportError:  # Graceful fallback if IceCream isn't installed.
    ic = lambda *a: None if not a else (a[0] if len(a) == 1 else a)  # noqa


from common import (
    Base, return_test_engine, LabelledDistogram, make_distribution, 
    make_distogram, AggregationType, delete_tables, taxi_data_headers_map)

debug = False

try:
    import zoneinfo
except ImportError:
    from backports import zoneinfo

    
    
project_id = os.environ.get('DEVSHELL_PROJECT_ID')
dataset = 'default_dataset'
sleep = 1.0  # seconds
batch_size = 10000

# store histograms, time, table, project, 


def prune_test_data(engine):
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()

    aggregation_lengths = [120, 120, 48, 60, 24, 10]
    aggregation_labels = (
        ["seconds", "minutes", "hours", "days", "months", "years"])

    # load initial data
    labelled_distogram_list = list(
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.datetime)) 
    labelled_distogram_hash = {
        instance.primary_key: instance for instance in labelled_distogram_list}

    df_metadata = pd.DataFrame(
        [instance.metadata_list for instance in labelled_distogram_list],
        columns=labelled_distogram_list[0].metadata_labels)

    ic(df_metadata.columns)
    if is_interactive():
#         display(df_metadata)
        pass
    first_time = df_metadata.iloc[0].at['datetime']
    last_time = df_metadata.iloc[-1].at['datetime']
    df_metadata['parsed'] = False
    
    # combine seconds to minutes
    # @TODO: writing time rounding function
    aggregation_index = 0
    df_fine = df_metadata[
        df_metadata['aggregation_type'] == aggregation_labels[aggregation_index]]
    
    delta_dict = {
        aggregation_labels[aggregation_index]: aggregation_lengths[aggregation_index]}
    min_keep_time = last_time - relativedelta(**delta_dict)

    delta_dict = {aggregation_labels[aggregation_index + 1]: 1}

    merge_stop_time = last_time + relativedelta(seconds = 1)
#     while merge_stop_time > first_time:
#     if True:
#     ic(len(df_fine))
    for i in range(10):
        merge_start_time = merge_stop_time - relativedelta(**delta_dict)

        rounding_dict = {"microsecond": 0}
        for j in range(aggregation_index + 1):
            rounding_dict[aggregation_labels[j][:-1]] = 0
        merge_start_time = merge_start_time.replace(**rounding_dict)
#         ic(i)
#         ic(merge_start_time)
#         ic(merge_stop_time)
        
        df_mask = ((df_fine["datetime"] < merge_stop_time)
                    & (df_fine["datetime_min"] >= merge_start_time))
        df_merge = df_fine[df_mask]
        df_fine.loc[df_mask, "parsed"] = True
        ic(len(df_merge))

        if is_interactive():
            pass
#             display(df_merge)
        if len(df_mask) > 0:
            merge_stop_time = merge_start_time
#             ic()
        else:
            merge_stop_time = df_fine[df_fine["parsed"] == False].iloc[-1].at['datetime']
            merge_stop_time = merge_stop_time + relativedelta(seconds = 1)
            ic()
            
#     df_merge = df_metadata[(
#         df_metadata['datetime']]

# df1 = df[(df.a != -1) & (df.b != -1)]

def publish_pruned_test_data(engine):
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    if True:
        print("adding pruned_test data")
        aggregation_lengths = [120, 120, 48, 60, 24, 10]
        aggregation_labels = (
            ["seconds", "minutes", "hours", "days", "months", "years"])
        date0 = datetime(2021, 10, 31)
        for i in range(len(AggregationType)):
        # i = 0
        # if True:
            print(AggregationType(i).name)
            j_min = 0
            j_max = 5 * aggregation_lengths[i]
            j_delta = aggregation_lengths[i] // 5
            last_date = date0
            for j in range(j_min, j_max, j_delta):
                delta_dict = {aggregation_labels[i]: -j}
                delta = relativedelta(**delta_dict)
                date = date0 + delta
                # America/New_York
                date.replace(tzinfo=zoneinfo.ZoneInfo('Etc/UTC'))
                h = make_distogram(make_distribution())
                d = LabelledDistogram(
                    data_source="debug3",
                    variable_name="x",
                    datetime_min=last_date,
                    datetime=date,
                    aggregation_type=AggregationType(0).name,
                    temporary_record=False,
                    distogram=h)
                session.add(d)
                session.commit()
                last_date = date
        print("before commit")
        session.commit()
        print("after commit")


def publish_taxi_data(engine):

    def publish_single_time():
        for header, my_distogram in distogram_dict.items():
            d = LabelledDistogram(
                data_source="taxi_min_max",
                variable_name=header,
                datetime=max_datetime,
                aggregation_type=aggregation_type,
                distogram=my_distogram)
            session.add(d)
            session.commit()
        print(f"published data: {max_datetime}")
        print(f"data_source: {d.data_source}")   
             
    Base.metadata.create_all(engine)
    Session = sessionmaker(bind=engine)
    session = Session()
    print("taxi!")
    # minutes/hours/days/weeks
    df = pd.read_parquet('data.parquet')
    df.sort_values(by=['tpep_dropoff_datetime'])
    # df2 = df.tail(1000000)['dropoff_latitude']
    # df2.to_csv('latitude.csv')
    # sys.exit()
    aggregation_dict = {
        # "minutes": 15, 
        # "hours": 1, 
        "days": 1}
    for aggregation_type, aggregation_length in aggregation_dict.items():

        delta_time_dict = {aggregation_type: aggregation_length}
        delta_time = relativedelta(**delta_time_dict)
        dt0 = df.iloc[0]['tpep_dropoff_datetime']
        print(f"dt0: {dt0}")
        if aggregation_type in {"minutes", "hours"}:
            dt0 = datetime(*dt0.timetuple()[:4])  # year/month/day/hour
        else:
            dt0 = datetime(*dt0.timetuple()[:3])  # year/month/day
        print(f"dt0: {dt0}")
        min_datetime = dt0
        max_datetime = dt0 + delta_time
        create_new_distograms = True
        for index, row in df.tail(1000000).iterrows():
            if create_new_distograms:
                distogram_dict = {
                    header: distogram.Distogram(with_min_max_list=True) 
                    for header in taxi_data_headers_map
                    if header != 'tpep_dropoff_datetime'}
                create_new_distograms = False
            for header, my_distogram in distogram_dict.items():
                if debug:
                    print(f"json pickle")
                    # print(f"empty: {jsonpickle.encode(my_distogram, indent=2)}")
                    my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                    # print(f"None max_min: {jsonpickle.encode(my_distogram, indent=2)}")
                    my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                    print(f"Row max_min: {jsonpickle.encode(my_distogram, indent=2)}")
                    # print(f"Row.to_dict: {jsonpickle.encode(row.to_dict(), indent=2)}")
                    sys.exit()
                # my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
                my_distogram = distogram.update(my_distogram, row[header], obj=row.to_dict())
            if row['tpep_dropoff_datetime'] > max_datetime:
                create_new_distograms = True 
                publish_single_time()
                min_datetime = max_datetime
                max_datetime = max_datetime + delta_time
        publish_single_time()


def main(engine):

    print("before create")
    Base.metadata.create_all(engine)

    Session = sessionmaker(bind=engine)
    session = Session()

    print("before read")
    success = False
    for instance in (
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.primary_key)):
        success = True
        # print(instance.primary_key, instance.variable_name, instance.datetime)


    if success:
        print()
        print(instance.primary_key, instance.variable_name)
        h2 = jsonpickle.decode(instance.distogram_string)
        print(f"min/mean/max {h2.min}/{distogram.mean(h2)}/{h2.max}")



    # publish_taxi_data(engine)
    # every 
    print("read from database")
    for instance in (
        session.query(LabelledDistogram).order_by(
            LabelledDistogram.primary_key)):
        pass
    print(
        instance.primary_key,
        instance.variable_name,
        instance.aggregation_type,
        instance.data_source,
        instance.datetime)
    
        # publish_pruned_test_data(engine)
#     prune_test_data(engine)

# from datetime import date
# from dateutil.relativedelta import relativedelta

# six_months = date.today() + relativedelta(months=+6)


if __name__ == "__main__":
    import os
    database_list = [
        "bigquery", "sqlite-memory", "sqlite-disk", "sqlite-disk-2", "postgres"]
    database = database_list[3]
    engine = return_test_engine(database)
    print(engine)

    # test(engine)
#     main(engine)
#     os.remove("./localdb-2")
#     publish_pruned_test_data(engine)

    prune_test_data(engine)


Engine(sqlite:///./localdb-2)
2021-12-19 06:45:23,215 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-19 06:45:23,216 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("distograms")
2021-12-19 06:45:23,218 INFO sqlalchemy.engine.Engine [raw sql] ()
2021-12-19 06:45:23,219 INFO sqlalchemy.engine.Engine COMMIT
2021-12-19 06:45:23,221 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2021-12-19 06:45:23,223 INFO sqlalchemy.engine.Engine SELECT distograms.primary_key AS distograms_primary_key, distograms.data_source AS distograms_data_source, distograms.variable_name AS distograms_variable_name, distograms.datetime AS distograms_datetime, distograms.datetime_min AS distograms_datetime_min, distograms.distogram_string AS distograms_distogram_string, distograms.aggregation_type AS distograms_aggregation_type, distograms.temporary_record AS distograms_temporary_record 
FROM distograms ORDER BY distograms.datetime
2021-12-19 06:45:23,225 INFO sqlalchemy.engine.Engine [generated in 0.001

ic| df_metadata.columns: Index(['primary_key', 'data_source', 'variable_name', 'datetime',
                                'datetime_min', 'aggregation_type', 'temporary_record', 'mean', 'stdev',
                                'max', 'min'],
                               dtype='object')
ic| len(df_merge): 14
ic| len(df_merge): 9
ic| len(df_merge): 8
ic| len(df_merge): 9
ic| len(df_merge): 8
ic| len(df_merge): 9
ic| len(df_merge): 8
ic| len(df_merge): 9
ic| len(df_merge): 8
ic| len(df_merge): 7


In [ ]:
#gcp_app.py
#######################
# notes
#######################

# Future development/polishing:
# track representative points for each histogram bin as well as min/max

# use uuid again?
# object dict

# consider logarithmic time axis: 
# https://community.plotly.com/t/plotting-time-dates-on-a-log-axis/33205/7

# time series from postgres; histogram from time series point
# see "shiny_example_2"

# MVP: time series/histogram/extrema/integration/table
# 12: extrema 1; 6/1/4/1

# dash plan:
# row 0 controls
# row 1 min/histogram/max
# row 2 time series/table

# consider row 1 time series/histogram

# controls: integrated/discrete; variable

# functions: select data

# dtructure

#######################
# python imports
#######################

# system
import json
import math

from dotenv import load_dotenv

# community
import distogram
import numpy as np
import pandas as pd

from sqlalchemy.orm import sessionmaker

# plotly/dash
import dash
import plotly.express as px
from dash import dcc
from dash import html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import plotly.graph_objs as go
from dash import dash_table as dt

# local
from common import (
    LabelledDistogram, return_test_engine, histogram_step_plot_data)


def is_interactive():
    import __main__ as main
    return not hasattr(main, '__file__')

if is_interactive():
    print("interactive!")
    from jupyter_dash import JupyterDash
    
# # conditional imports
# try:
#     import zoneinfo
# except ImportError:
#     from backports import zoneinfo

#######################
# Debug
#######################


#######################
# Data Analysis / Model
#######################

# choose database (currently postgres)
load_dotenv()
database_list = [
    "bigquery", "sqlite-memory", "sqlite-disk", "postgres"]
database = database_list[2]
engine = return_test_engine(database)

# load initial data
Session = sessionmaker(bind=engine)
session = Session()
labelled_distogram_list = list(
    session.query(LabelledDistogram).order_by(
        LabelledDistogram.datetime)) 
labelled_distogram_hash = {
    instance.primary_key: instance for instance in labelled_distogram_list}

df_metadata = pd.DataFrame(
    [instance.metadata_list for instance in labelled_distogram_list],
    columns=labelled_distogram_list[0].metadata_labels)


#########################
# Dashboard Layout / View
#########################

def gcp_control(label, choices):
    return dbc.Card(
        [html.Div(
            [dbc.Label(label.replace("_", " ")),
             dcc.Dropdown(
                id=label,
                options=[
                    {"label": item, "value": item} for item in choices],
                value=choices[0],
                searchable=False,
            )])
         ]
    )

if is_interactive():
    app = JupyterDash(external_stylesheets=[dbc.themes.BOOTSTRAP])
else:
    app = dash.Dash(external_stylesheets=[dbc.themes.BOOTSTRAP])

app.layout = dbc.Container(
    [
        html.H1('{Streaming} summary statistics'),
        html.Hr(),
        dbc.Row(
            [
                dbc.Col(
                    gcp_control(
                        'data_source', 
                        df_metadata['data_source'].unique()
                    ), md=3),
                dbc.Col(
                    gcp_control(
                        'variable_name', 
                        df_metadata['variable_name'].unique()
                    ), md=3),

                dbc.Col(
                    gcp_control(
                        'aggregation_type', 
                        df_metadata['aggregation_type'].unique()
                    ), md=3),
            ],
            align='center',
        ),        
        dbc.Row(
            [
                dbc.Col(dcc.Graph(id='time-series-graph'), md=5),
                dbc.Col([dbc.Col(id='min-table')], md=1),
                dbc.Col(dcc.Graph(id='histogram-graph'), md=5),
                dbc.Col([dbc.Col(id='max-table')], md=1),
            ],
            align='center',
        ),
        # dbc.Row(
        #     [
        #         dbc.Col([dbc.Col(id='detail-table')], md=12),
        #     ],
        #     align='center',
        # ),
        dbc.Row(
            [
                dbc.Col(html.Pre(id='click-data'), md=4),
                dbc.Col(html.Pre(id='click-data-2'), md=4),
            ],
            align="center",
        ),
    ],
    fluid=True,
)


#############################################
# Interaction Between Components / Controller
#############################################

def get_instance(data_source, variable_name, aggregation_type, clickData):
    ctx = dash.callback_context
    trigger = ctx.triggered[0]['prop_id'].split('.')[0]

    if clickData == None or trigger != 'time-series-graph':
        df = df_metadata
        df_subset = df.loc[
            (df['data_source'] == data_source) 
            & (df['variable_name'] == variable_name)
            & (df['aggregation_type'] == aggregation_type)]
        instance = labelled_distogram_hash[df_subset.iloc[-1].primary_key]
    else:
        instance = labelled_distogram_hash[
            clickData['points'][0]['customdata']]
    return instance


@app.callback(
    Output('time-series-graph', 'figure'),
    [
        Input('data_source', 'value'),
        Input('variable_name', 'value'),
        Input('aggregation_type', 'value'),
    ],
)
def make_time_series_graph(data_source, variable_name, aggregation_type):
    df = df_metadata
    df_subset = df.loc[
        (df['data_source'] == data_source) 
        & (df['variable_name'] == variable_name)
        & (df['aggregation_type'] == aggregation_type)]
    # Create figure
    fig = go.Figure()
    x_label = 'datetime'
    variable_list = ['max', 'mean', 'min']
    # Loop df columns and plot columns to the figure
    for variable in variable_list:
        fig.add_trace(go.Scatter(
            x=df_subset[x_label], 
            y=df_subset[variable],
            customdata=df_subset['primary_key'],
            mode='markers',  # 'lines' or 'markers'
            name=variable))
    fig.update_layout(clickmode='event+select')
    fig.update_xaxes(title_text="date/time")
    fig.update_yaxes(title_text=variable_name)
    return fig


@app.callback(
    Output('histogram-graph', 'figure'),
    [
        Input('data_source', 'value'),
        Input('variable_name', 'value'),
        Input('aggregation_type', 'value'),
        Input('time-series-graph', 'clickData')
    ],
)
def make_histogram(data_source, variable_name, aggregation_type, clickData):
    instance = get_instance(
        data_source, variable_name, aggregation_type, clickData)
    hist = distogram.frequency_density_distribution(instance.distogram)
    if hist == None:
        fig = px.scatter(
            x=[instance.distogram.bins[0][0]],
            y=[1],
            labels={"x": variable_name, "y": "count"}
            )
        return fig
    labels = ["bin", "count/bin width"]
    df_hist = histogram_step_plot_data(hist, labels)
    fig = px.line(
        df_hist,
        x=labels[0],
        y=labels[1],
        labels = {labels[0]: variable_name},
        log_y=True,
    )
    fig.update_traces(mode="lines", line_shape="hv")
    fig.update_layout(clickmode='event+select')
    return fig

def make_min_max_table(
    data_source, variable_name, aggregation_type, clickData,
    min_max_type):
    if min_max_type not in ["min", "max"]:
        raise ValueError(
            f"{min_max_type} is not supported. "
            f"Only 'min' and 'max' are supported.")
    instance = get_instance(
        data_source, variable_name, aggregation_type, clickData)
    if min_max_type == "min":
        min_max_data = distogram.min_list(instance.distogram)
    elif min_max_type == "max":
        min_max_data = distogram.min_list(instance.distogram)
    # calculate necessary significant figures
    if len(min_max_data) > 1:
        min_data = min(min_max_data)
        max_data = max(min_max_data)
        range_list = [min_max_data[i] - min_max_data[i-1] for i in range(1,len(min_max_data))]
        range_list.sort()
        range_list = [i for i in range_list if i > 0]
        max_abs_data = max([abs(min_data), abs(max_data)])
        sig_figs_needed = -int(math.log10(range_list[0]/max_abs_data))
        format_string = "{" + f":.{sig_figs_needed}e" + "}"
    else:
        format_string = "{f:.2e}"
        
    df = pd.DataFrame(
        [format_string.format(i) 
                for i in min_max_data],
        columns = [min_max_type])
    return [dt.DataTable(
        columns=[{"name": i, "id": i} for i in df.columns],
        data=df.to_dict('records'),
        style_cell={'fontSize': 12}
    ),
    ]

@app.callback(
    Output('min-table', 'children'),
    [
        Input('data_source', 'value'),
        Input('variable_name', 'value'),
        Input('aggregation_type', 'value'),
        Input('time-series-graph', 'clickData')
    ],
)
def make_min_table(data_source, variable_name, aggregation_type, clickData):
    return make_min_max_table(
        data_source, variable_name, aggregation_type, clickData, min_max_type="min")

@app.callback(
    Output('max-table', 'children'),
    [
        Input('data_source', 'value'),
        Input('variable_name', 'value'),
        Input('aggregation_type', 'value'),
        Input('time-series-graph', 'clickData')
    ],
)
def make_max_table(data_source, variable_name, aggregation_type, clickData):
    return make_min_max_table(
        data_source, variable_name, aggregation_type, clickData, min_max_type="max")

# @app.callback(
#     Output('detail-table', 'children'),
#     [
#         Input('data_source', 'value'),
#         Input('variable_name', 'value'),
#         Input('aggregation_type', 'value'),
#         Input('time-series-graph', 'clickData'),
#         Input('histogram-graph', 'clickData')
#     ],
# )
# def make_max_table(data_source, variable_name, aggregation_type, clickData):
#     instance = get_instance(
#         data_source, variable_name, aggregation_type, clickData)
#     return [dt.DataTable(
#         data=[{'max': "{:.2e}".format(instance.distogram.max)}],
#         columns=[{'name': 'max', 'id': 'max'}],
#         style_cell={'fontSize': 12}
#     ),
#     ]

@app.callback(
    Output('click-data', 'children'),
    Input('time-series-graph', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


@app.callback(
    Output('click-data-2', 'children'),
    Input('histogram-graph', 'clickData'))
def display_click_data(clickData):
    return json.dumps(clickData, indent=2)


#############################################
# Interaction Between Components / Controller
#############################################


# start Flask server
# if __name__ == '__main__':
#     app.run_server(
#         debug=True,
#         host='0.0.0.0',
#         port=8050
#     )

if __name__ == '__main__':
    if is_interactive():
        app.run_server(mode='inline')
    else:
        app.run_server(debug=True, port=8891)
    

In [ ]:
#
from jupyter_dash import JupyterDash

import dash
import dash_table
import pandas as pd

df = pd.read_csv('https://raw.githubusercontent.com/plotly/datasets/master/solar.csv')

app = JupyterDash(__name__)

app.layout = dash_table.DataTable(
    id='table',
    columns=[{"name": i, "id": i} for i in df.columns],
    data=df.to_dict('records'),
)

if __name__ == '__main__':
    app.run_server(mode='inline')